# 1. Dataset Introduction & Loading

The dataset `short_seasonal.csv` is a synthetic dataset, data generation code can be found in `src/synthetic_data_generation.ipynb`. It is a small dataset of 80 data points and simulates monthly data with strong seasonality and a positive linear trend. 

The purpose of this dataset is to provide a small, computationally efficient dataset to test upon. It's short length and strong seasonality also brings unique challenges to data-intensive models with less seasonality adaptions.

### Loading data from `csv`

In [ ]:
import pandas as pd

df = pd.read_csv('./data/short_seasonal.csv')

df['timestamp'] = pd.to_datetime(df['timestamp'])
ts = df.set_index("timestamp")["value"]
ts = ts.asfreq('ME')
lables = df.set_index("timestamp")["labels"]
lables = lables.asfreq('ME')

In [ ]:
from src.visualization.plots import plot_ts_with_anom

plot_ts_with_anom(ts, lables)

This time series is generated additive model, using a linear trend, period 12 seasonality, and $MA(1)$ noise.

The anomalies for this dataset occur from Feb 2019 to Dec 2019. The anomaly values are generated using independent sampling from a normal distribution with `mean=ts.mean()` and `std=2.5`.

For the upcoming modeling sections, I will be working only with the original time series **without** the labels. The goal of this report is to investigate how well models detect anomalies on **unlabeled** data.

# 2. Modeling - Classical Statistics

To begin, lets plot the time series to gain an overall understanding of it.

In [ ]:
from src.visualization.plots import plot_ts

plot_ts(ts)

Observations:
- Strong yearly seasonality, values peek during the first 1-2 months every year
- Increasing trend, seems roughly linear

## 2.1 STL Model

To begin, lets start with a quick Seasonal-Trend decomposition.

In [ ]:
from statsmodels.tsa.seasonal import STL

stl = STL(ts, period=12) 
stl_result = stl.fit()

_ = stl_result.plot()

Observations:
- Mostly linear trend with a light wave shape from 2017-2020
- Larger residuals from late 2018 to late 2019

### Investigation: Linear Trend

In [ ]:
from src.models.statistical.linear_models import linear_model
from src.visualization.plots import plot_fit

model, X = linear_model(stl_result.trend)
lin_results = model.fit()

lin_pred = pd.Series(lin_results.predict(X), index=ts.index)
plot_fit(stl_result.trend, lin_pred, title='Linear model fit to Trend')

print(lin_results.summary())

The summary provides evidence that trend should be modeled linearly. With a $p$-value of $0.000$ for all parameters, there is strong evidence against any model parameters being zero, hence no need to drop parameters. The R-squared and Adj. R-squared are both high at $0.968$, showing $96.8\%$ of the response can be explained by the explanatory variables

In [ ]:
from src.visualization.plots import plot_resid

lin_resid = pd.Series(lin_results.resid, index=ts.index)
plot_resid(lin_resid)

There is strong trend in the residual plot suggesting that the residuals are not indenpent and identiacally distributed normal. The original time series deviates futhest from the preditions from late 2018 to late 2019 $(|r_i|>0.5)$, suggesting that this may be where the anomalies occur.

### Investigation: Residuals

In [ ]:
large_STL_resid = (abs(stl_result.resid) > 2)
plot_resid(stl_result.resid, hlines=[-2, 2])

The residuals look independent and evenly spread across zero, with outliers occuring between mid 2018 and late 2019.

In [ ]:
from src.models.statistical.rule_based_functions import cusum

# Add plot cusum sum option
cusum_resid = cusum(stl_result.resid, stl_result.resid.mean(), threshold=(3 * stl_result.resid.std()), drift=(0.1 * stl_result.resid.std()))
plot_ts_with_anom(ts, cusum_resid)

No significant anomaly clusters are found by running CUSUM on the residuals, showing little change in mean of the residuals.

## 2.2 SARIMA Model
### Differenced Time Series

Base on part 2.1, it is safe to assume that there is a seasonal component of period 12 and a linear trend.

In [ ]:
ts_diff = ts.diff(12).diff()
plot_ts(ts_diff)

Differencing the time series first by the period (to remove seasonalitiy) then by 1-step back in time give the above graph. This differenced times series looks to have constant mean around zero, but with variablilty in variance.

In [ ]:
# Sliding window variance

In [ ]:
from statsmodels.tsa.stattools import adfuller

stationary_pval = adfuller(ts_diff.dropna())[1]
print(f'p-value: {stationary_pval}')

Applying the augmented Dickey–Fuller test on the differenced time series returns a statistically significant $p$-value, thus there is strong evidence that the time series is stationary (constant mean and variance).

### SARIMA Model selection

Lets begin by plotting the acf/pacf values to get an idea of the auto-correlation for this times series.

In [ ]:
from statsmodels.tsa.stattools import acf, pacf
from scipy.stats import norm
from src.visualization.plots import plot_lag_with_ci
import numpy as np

nlags = 24 # Covers 2 periods
pacf_vals = pacf(ts, nlags=nlags)
acf_vals = acf(ts, nlags=nlags, fft=True)

conf_interval = norm.ppf(1 - 0.05 / 2) / np.sqrt(len(ts))
plot_lag_with_ci(pacf_vals, conf_interval, title='PACF')
plot_lag_with_ci(acf_vals, conf_interval, title='ACF')

Both the ACF and PACF plots have a notably high lag at $12$, suggesting $(P, D, Q)=(1, 1, 1)$. 

The ACF and PACF both seem to be deminished in value over higher lags, but have many statistically significant lags in the first period $(< 12)$. This may be due to small data sample size (total length of $80$ and post-differencing lenght of $67$). For simplicity, only $p, q < 3$ are considered.

In [ ]:
from src.models.statistical.stationary_models import SARIMA_grid_search

results_table = SARIMA_grid_search(ts, period=12, d=1, D=1, max_p=3, max_q=3, max_P=1, max_Q=1)
df_results = pd.DataFrame(results_table).sort_values('AIC', na_position='last')

display(df_results[['order', 'seasonal_order', 'AIC', 'BIC']].head(10))

The top 10 models show no significant difference in their `aic` or `bic` scores. Taking both metrics and model complexcity into consideration, I have choosen $SARIMA(0, 1, 1)(0, 1, 1, 12)$ to be the model for further analysis.

The choosen model is plotted below.

In [ ]:
best_model = df_results.iloc[3]
plot_fit(ts, fitted_vals=best_model['model'].fittedvalues[13:], CI=best_model['conf_int'], anom=best_model['anom'], title=f'SARMIA{best_model['order']}{best_model['seasonal_order']}')

### SARIMA residual analysis

In [ ]:
SARIMA_resid = best_model['model'].resid[13:]
plot_resid(SARIMA_resid)

In [ ]:
# Do rolling window variance + outlier cutoff

### Across model Anomalies

Going back to all the SARIMA models fitted during the grid search, we can look at the data points where all models had a hard time predicting

In [ ]:
from src.models.statistical.stationary_models import count_anoms

anom_counts = count_anoms(df_results[['anom']].head(10))
plot_ts(anom_counts)

In [ ]:
anom_counts = count_anoms(df_results[['anom']])
plot_ts(anom_counts)

This shows that models have a significatly harder time predicting data points from early 2019 to early 2020

## 2.3 BOCPD

## 2.4 Kalman Filters

# 3 Self Trained Machine Learning models

Add Isolation forest and TCN

## 3.1 LSTM Autoencoder

In [ ]:
import torch
from src.models.preprocessing import create_sliding_windows, np_to_dataloader

g = torch.Generator()
g.manual_seed(13)

# Add preprocessing
windows = create_sliding_windows(ts, window_size=12)
data_loader = np_to_dataloader(windows, batch_size=16, generator=g)

In [ ]:
from src.models.self_trained_ml.lstm_ae import LSTMAutoencoder
import torch.optim as optim
import torch.nn as nn

model = LSTMAutoencoder(hidden_dim=8, latent_dim=8)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-1)

In [ ]:
from src.models.self_trained_ml.lstm_ae import train_LSTMAE, eval_LTSMAE_MSE
import torch

train_LSTMAE(model, data_loader, criterion=criterion, optimizer=optimizer, epochs=50)
mse_LSTM = eval_LTSMAE_MSE(model, test_data=torch.tensor(windows, dtype=torch.float32).unsqueeze(-1))

errors_ts = pd.Series(mse_LSTM.numpy(), index=ts.index[:len(ts)-12])
plot_ts(errors_ts)

# 4 Pretrained Models

## 4.1 Prophet

In [ ]:
from prophet import Prophet

prophet_df = pd.DataFrame({
    'ds': ts.index,  
    'y': ts.values 
})

model = Prophet()
model.fit(prophet_df)

prophet_pred = model.predict(prophet_df[['ds']].copy())

prophet_anom = (prophet_df['y'] < prophet_pred['yhat_lower']) | (prophet_df['y'] > prophet_pred['yhat_upper'])
prophet_anom = pd.Series(prophet_anom.values, index=ts.index)

prophet_CI = prophet_pred[['yhat_lower', 'yhat_upper']].copy()
prophet_CI.index = ts.index

plot_fit(ts, pd.Series(prophet_pred['yhat'].values, index=ts.index), CI=prophet_CI, anom=prophet_anom)

## 4.2 AWS Lookout for Metrics

Due to the [data size and constraint requirments](https://docs.aws.amazon.com/lookoutmetrics/latest/dev/detectors-setup.html#:~:text=Detector%20Statuses,data%20requirements.) for the AWS Lookout for Metrics model. 

AWS_df = df[['timestamp', 'value']].rename(columns={"value": "metric_value"})
print(AWS_df)

AWS_csv = './data/AWS_LfM_csv/short_seasonal_AWS.csv'
AWS_df.to_csv(AWS_csv)

import boto3

s3 = boto3.client('s3', region_name='us-east-1')
bucket_name = "time-series-anomaly-project" 

s3.upload_file(AWS_csv, bucket_name, 'short_seasonal_AWS.csv')

client = boto3.client('lookoutmetrics', region_name='us-east-1')


detector_arn = response['AnomalyDetectorArn']
client.activate_anomaly_detector(AnomalyDetectorArn=detector_arn)

client.describe_anomaly_detector(AnomalyDetectorArn=detector_arn)

response = client.get_anomaly_group(
    AnomalyGroupId='your-group-id',
    AnomalyDetectorArn=detector_arn
)